## 329. Longest Increasing Path in a Matrix
- Description:
  <blockquote>
    Given an `m x n` integers `matrix`, return  *the length of the longest increasing path in* `matrix`.
   
  From each cell, you can either move in four directions: left, right, up, or down. You **may not** move **diagonally** or move **outside the boundary** (i.e., wrap-around is not allowed).
   
  **Example 1:**
  ![Image](https://assets.leetcode.com/uploads/2021/01/05/grid1.jpg)
   
  **Input:** matrix = `[9, 9,4],[6,6,8],[2,1,1]`
  **Output:** 4
  **Explanation:** The longest increasing path is `[1, 2, 6, 9]`.
   
  **Example 2:**
  ![Image](https://assets.leetcode.com/uploads/2021/01/27/tmp-grid.jpg)
   
  **Input:** matrix = `[3, 4,5],[3,2,6],[2,2,1]`
  **Output:** 4
  **Explanation: **The longest increasing path is `[3, 4, 5, 6]`. Moving diagonally is not allowed.
   
  **Example 3:**
  **Input:** matrix = [[1]]
  **Output:** 1
   
  **Constraints:**
   
  - `m == matrix.length`
  - `n == matrix[i].length`
  - `1 <= m, n <= 200`
  - `0 <= matrix[i][j] <= 231- 1`
  </blockquote>

- URL: [Problem_URL](https://leetcode.com/problems/longest-increasing-path-in-a-matrix/description/)

- Topics: Problem_topic

- Difficulty: Medium / Hard

- Resources: example_resource_URL

### Solution 1, DFS + Memoization / DP using 2D matrix
Solution description
- Time Complexity: O(M*N)
  - Each cell computed exactly once (cached after first visit)
  - 4 directions checked per cell → O(4mn) = O(mn)
- Space Complexity: O(M*N)
  - dp array: m × n
  - Recursion stack: Worst case O(mn) for longest path (e.g., spiral)

In [ ]:
class Solution:
    def longestIncreasingPath(self, matrix: List[List[int]]) -> int:
        if not matrix or not matrix[0]:
            return 0
        
        rows, cols = len(matrix), len(matrix[0])
        directions = ((1,0), (-1,0), (0,1), (0,-1))
        dp = [[-1] * cols for _ in range(rows)] # -1 indicates unvisited cells
        
        # dfs(r, c) = longest increasing path starting from cell (r, c)
        def dfs(r, c):
            if dp[r][c] != -1:
                return dp[r][c]
            
            max_len = 1
            
            for dr, dc in directions:
                nr, nc = r + dr, c + dc
                if is_valid_path(nr, nc, r, c):
                    # adding 1 for the current cell
                    # when you call dfs(nr, nc), you get the longest path from the neighbor onward, but it doesn't include the current cell you're standing on!
                    max_len = max(max_len, 1 + dfs(nr, nc))
                    
            dp[r][c] = max_len
            return max_len
        
        def is_valid_path(nr, nc, r, c):
            return 0 <= nr < rows and 0 <= nc < cols and matrix[nr][nc] > matrix[r][c]
        
        result = 0
        
        for i in range(rows):
            for j in range(cols):
                result = max(result, dfs(i, j))
        
        return result

### DFS + DP using Dict / Hash Map approach

1. Data Structure & Efficiency

    DP Matrix: Uses a fixed-size 2D array (dp[rows][cols]).
        Access: Direct indexing dp[r][c] is extremely fast (O(1)).
        Memory: Allocates all memory upfront. Since every cell in the matrix will likely be visited, this is very efficient for "dense" grids.
    DP Hash Map: Uses a dictionary (position_longest_path).
        Access: Average O(1), but requires hashing the tuple (i, j) and handling potential collisions.
        Memory: Grows dynamically. In Python, dictionaries have higher memory overhead per entry compared to a list of lists.

2. Performance in Python

    In LeetCode environments, the Matrix approach is generally faster. Hashing a tuple (i, j) on every recursive call adds overhead that accumulates, especially for larger matrices (e.g., 200×200).
    The Matrix approach also benefits from better cache locality, as the data is stored in contiguous blocks of memory.

3. Use Case Suitability

    Matrix: Best when the bounds are known and the "state space" is dense (i.e., you expect to fill most of the cache).
    Hash Map: Useful if the coordinates were sparse or if the range of i and j was extremely large but only a few coordinates were actually reachable.


In [ ]:
class Solution:
    def longestIncreasingPath(self, matrix: List[List[int]]) -> int:
        position_longest_path = {}
        rows = len(matrix)
        cols = len(matrix[0])
        longest_path = 0
        directions = ((1,0),(-1,0),(0,1),(0,-1))

        def is_valid_index(i, j):
            return 0 <= i < rows and 0 <= j < cols

        def dfs(i, j):
            path = 1
            
            if (i, j) in position_longest_path:
                return position_longest_path[(i,j)]

            for dr, dc in directions:
                nr, nc = i+dr, j+dc
                if is_valid_index(nr, nc) and matrix[nr][nc] > matrix[i][j]:
                    path = max(path, 1+dfs(nr, nc))
            
            position_longest_path[(i, j)] = path
            return path

        for i in range(rows):
            for j in range(cols):
                if (i,j) in position_longest_path:
                    longest_path = max(longest_path, position_longest_path[(i,j)])
                else:
                    longest_path = max(longest_path, dfs(i,j))
        
        return longest_path

### Solution 2, BFS + Topological Sort, AKA Peeling Onion
Benefit of this approach is that it is iterative so you do not need to worry about recursion limits and stack overflows

Instead of going top-down (DFS from each cell), go bottom-up:
- Start from "sink" cells (cells with no increasing neighbors)
- Gradually "peel" layers outward like an onion
- Track depth as you go

Think of the matrix as a Directed Acyclic Graph (DAG):

- Each cell is a node
- Edge from cell A → cell B if B's value > A's value (increasing)

Key Insight: We process cells in "reverse" order—from endpoints (high values) back to starting points (low values), counting layers as we peel.

Out-degree = number of neighbors with larger values (where you can go FROM this cell)

By starting from the "peaks" (cells with an out-degree of 0) and working backwards to the "valleys," you are effectively counting the number of levels in the topological sort, which corresponds to the length of the longest path

In your logic:

  1. Out-degree 0 means there are no neighbors with a strictly greater value. This cell is a "peak" or a local maximum for an increasing path.
  2. When you pop it, you look for neighbors with smaller values. By decrementing their out-degree, you are effectively saying: "One of the possible paths upward from this smaller neighbor has been fully explored."
  3. When a smaller neighbor's out-degree hits 0, it means all paths starting from it towards larger values have been accounted for, and it is now ready to be categorized into the next "level" of the path.

One subtle detail: If a neighbor has the same value (e.g., 5 next to 5), it doesn't count toward the out-degree because the path must be strictly increasing. Your code correctly handles this since matrix[nr][nc] > matrix[r][c] would be false.


- Time Complexity: O(M*N)
  - Each cell computed exactly once
- Space Complexity: O(M*N)
  - queue + out-degree array

In [ ]:
from collections import deque


class Solution:
    def longestIncreasingPath(self, matrix: List[List[int]]) -> int:
        if not matrix or not matrix[0]:
            return 0
    
        rows, cols = len(matrix), len(matrix[0])
        directions = ((1,0), (-1,0), (0,1), (0,-1))
        
        out_degree = [[0] * cols for _ in range(rows)]
        
        def is_valid_index(nr, nc):
            return 0 <= nr < rows and 0 <= nc < cols
        
        # Step 1: Calculate out-degrees
        for r in range(rows):
            for c in range(cols):
                for dr, dc in directions:
                    nr, nc = r+dr, c+dc
                    
                    if is_valid_index(nr, nc) and matrix[nr][nc] > matrix[r][c]:
                        out_degree[r][c] += 1
        
        # Step 2: Initialize queue with cells having out-degree 0
        queue = deque()
        for r in range(rows):
            for c in range(cols):
                if out_degree[r][c] == 0:
                    queue.append((r, c))
        
        # Step 3: BFS layer by layer
        path_length = 0
        while queue:
            path_length += 1
            
            for _ in range(len(queue)):
                r, c = queue.popleft()
                
                # Check all neighbors with SMALLER values
                for dr, dc in directions:
                    nr, nc = r+dr, c+dc
                    
                    if is_valid_index(nr, nc) and matrix[nr][nc] < matrix[r][c]:
                        out_degree[nr][nc] -= 1
                        
                        if out_degree[nr][nc] == 0:
                            queue.append((nr, nc))
        
        return path_length
        

### Solution 3, Naive DFS [Time Limit Exceeded]
Solution description


- Time complexity : O(2^(m+n)). The search is repeated for each valid increasing path. In the worst case we can have O(2m+n) calls.

- Space complexity : O(mn). For each DFS we need O(h) space used by the system stack, where h is the maximum depth of the recursion. In the worst case, O(h)=O(mn).


In [ ]:
class Solution:
    def longestIncreasingPath(self, matrix: List[List[int]]) -> int:
        if not matrix or not matrix[0]:
            return 0
        
        rows, cols = len(matrix), len(matrix[0])
        directions = [(1,0), (-1,0), (0,1), (0,-1)]
        
        def dfs(r, c):
            max_len = 1
            for dr, dc in directions:
                nr, nc = r + dr, c + dc
                if validipath(nr, nc, r, c):
                    max_len = max(max_len, 1 + dfs(nr, nc))
            return max_len
        
        def validipath(nr, nc, r, c):
            return 0 <= nr < rows and 0 <= nc < cols and matrix[nr][nc] > matrix[r][c]
        
        result = 0
        
        for i in range(rows):
            for j in range(cols):
                result = max(result, dfs(i, j))
        
        return result